# Project Index

[Custom Model Notebook](../../notebooks/custom_model.ipynb)  
[Training Notebook](../../notebooks/train.ipynb)  
[Project Config Notebook](../../notebooks/project_config.ipynb)  
[Forgather Notebook](../../notebooks/forgather.ipynb)  

In [3]:
import forgather.nb.notebooks as nb

nb.display_project_index(show_pp_config=True, show_generated_code=True)

# Traning  for Fashion

This project reproduces the configuration from a PyTorch tutorial, where a simple ML model is created and trained to recognize categories of clothing from the FashionMNIST dataset.

https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

This was chosen as it is a relatively simple project which can be relativley self contained. Still, it is far more complex than the previous examples.

The model itself does not require any custom code. It's simply a stack of PyTorch layers, chained together with a nn.Sequential. If you would like to know more about the model itself, see:

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

## Custom Code

While Forgather is good at assembling objects, the language is not practical for defining logic. For this, we have defined a custom "trainer" class in the project's 'src' directory and we will use Forgather to dynamically import this code, injecting all the required dependencies.

Unlike the previous projects, you will note that the "Modules" section not empty and has a link to the Trainer definition.

## Project Structure

Like the previous example, this project makes use of template inheritance, where there is a common 'project.yaml' file from which all of the configuratioins are derived.

The template provides the basic structure, with 'blocks' which may be substituted or extended by child templates. We use this functionaity in the configurations to override various components of the base configuraiton.

This is still a relatively simple project, as it does not reference any external template libraries. We will get to that in the next example.

## Code Generation

Note the output of the code generator. It has detected the inclusion of a dynamic import, thus it has automatically defined a function for importing dynamic modules.

Also note that it knows how to translate some of the rather clunky expreressions from the original YAML file, like calling a method, into relatively clean Python code.

---



#### Project Directory: "/home/dinalt/ai_assets/forgather/tutorials/project_gamma"

## Meta Config
Meta Config: [/home/dinalt/ai_assets/forgather/tutorials/project_gamma/meta.yaml](meta.yaml)

- [meta.yaml](meta.yaml)

Template Search Paths:
- [/home/dinalt/ai_assets/forgather/tutorials/project_gamma/templates](templates)

## Available Configurations
- [wider.yaml](templates/experiments/wider.yaml)
- [deeper.yaml](templates/experiments/deeper.yaml)
- [adam.yaml](templates/experiments/adam.yaml)
- [baseline.yaml](templates/experiments/baseline.yaml)

Default Configuration: baseline.yaml

Active Configuration: baseline.yaml

## Included Templates
- [experiments/baseline.yaml](templates/experiments/baseline.yaml)
    - [project.yaml](templates/project.yaml)
        - [formatting.yaml](templates/formatting.yaml)
### Config Metadata:

```python
{'citation': 'https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html',
 'config_class': 'fashion_trainer',
 'config_description': 'Base configuration, based on Torch tutorial '
                       'parameters.',
 'config_name': 'Fashion MNIST Trainer'}

```

## Modules
- [./src/trainer.py](src/trainer.py) : Trainer
    - [/home/dinalt/ai_assets/forgather/tutorials/project_gamma/./src/trainer.py](src/trainer.py) : trainer
## Output Targets
- meta
- main
- args
- model_params
- model
- training_data
- test_data
- train_dataloader
- test_dataloader
- loss_fn
- optimizer

## Preprocessed Config

```yaml

#---------------------------------------
#          Fashion MNIST Trainer         
#---------------------------------------
# 2024-08-15T07:00:22
# Description: Base configuration, based on Torch tutorial parameters.
# Project Dir: /home/dinalt/ai_assets/forgather/tutorials/project_gamma
# Citation: https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
#---------------------------------------


############# Config Vars ##############

# ns.hidden_dim = 512
# ns.epochs = 5
# ns.batch_size = 64
# ns.lr = 0.001
# ns.logging_steps = 100



########### Model Definition ###########

.define: &activation_fn !factory:torch.nn:ReLU@activation_fn []

.define: &model !singleton:torch.nn:Sequential@model
    - !factory:torch.nn:Flatten []
    - !factory:torch.nn:Linear [ 784, 512 ]
    - *activation_fn
    - !factory:torch.nn:Linear [ 512, 512 ]
    - *activation_fn
    - !factory:torch.nn:Linear [ 512, 10 ]

############### Dataset ################

.define: &transform !factory:torchvision.transforms:ToTensor@transform []

.define: &training_data !singleton:torchvision.datasets:FashionMNIST
    root: "data"
    train: True
    download: True
    transform: *transform

.define: &test_data !singleton:torchvision.datasets:FashionMNIST
    root: "data"
    train: False
    download: True
    transform: *transform

.define: &train_dataloader !singleton:torch.utils.data:DataLoader
    args: [ *training_data ]
    kwargs: { batch_size: 64 }

.define: &test_dataloader !singleton:torch.utils.data:DataLoader
    args: [ *test_data ]
    kwargs: { batch_size: 64 }

############### Trainer ################

.define: &model_params !singleton:call [ !singleton:getattr [ *model, "parameters" ] ]

# **Optimizer**

.define: &optimizer !singleton:torch.optim:SGD
    args: [ *model_params ]
    kwargs:
        lr: 0.001

# **Loss Function**

.define: &loss_fn !singleton:torch.nn:CrossEntropyLoss []

# **Trainer**

.define: &trainer !singleton:./src/trainer.py:Trainer@trainer
    train_dataloader: *train_dataloader
    test_dataloader: *test_dataloader
    model: *model
    loss_fn: *loss_fn
    optimizer: *optimizer
    epochs: 5
    batch_size: 64
    logging_steps: 100

################ Output ################

meta:
    config_name: "Fashion MNIST Trainer"
    config_description: "Base configuration, based on Torch tutorial parameters."
    config_class: "fashion_trainer"
    citation: "https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html"

main: *trainer

args:
    hidden_dim: 512
    epochs: 5
    batch_size: 64
    logging_steps: 100
    lr: 0.001
model_params: *model_params
model: *model
training_data: *training_data
test_data: *test_data
train_dataloader: *train_dataloader
test_dataloader: *test_dataloader
loss_fn: *loss_fn
optimizer: *optimizer

```

## Generated Code

```python
from torch.nn import ReLU
from torchvision.transforms import ToTensor
from torch.nn import CrossEntropyLoss
from torch.nn import Flatten
from torch.utils.data import DataLoader
from torch.nn import Sequential
from torch.optim import SGD
from torch.nn import Linear
from torchvision.datasets import FashionMNIST
from importlib.util import spec_from_file_location, module_from_spec
import os
import sys

# Import a dynamic module.
def dynimport(module, name, searchpath):
    module_path = module
    module_name = os.path.basename(module).split(".")[0]
    module_spec = spec_from_file_location(
        module_name,
        module_path,
        submodule_search_locations=searchpath,
    )
    mod = module_from_spec(module_spec)
    sys.modules[module_name] = mod
    module_spec.loader.exec_module(mod)
    for symbol in name.split("."):
        mod = getattr(mod, symbol)
    return mod

Trainer = lambda: dynimport("./src/trainer.py", "Trainer", [])

def construct(
):
    transform = lambda: ToTensor()

    activation_fn = lambda: ReLU()

    model = Sequential(
        Flatten(),
        Linear(
            784,
            512,
        ),
        activation_fn(),
        Linear(
            512,
            512,
        ),
        activation_fn(),
        Linear(
            512,
            10,
        ),
    )

    trainer = Trainer()(
        train_dataloader=DataLoader(
            FashionMNIST(
                root='data',
                train=True,
                download=True,
                transform=transform(),
            ),
            batch_size=64,
        ),
        test_dataloader=DataLoader(
            FashionMNIST(
                root='data',
                train=False,
                download=True,
                transform=transform(),
            ),
            batch_size=64,
        ),
        model=model,
        loss_fn=CrossEntropyLoss(),
        optimizer=SGD(
            model.parameters(),
            lr=0.001,
        ),
        epochs=5,
        batch_size=64,
        logging_steps=100,
    )
    
    return trainer

```



## Construct Baseline Configuration

In [2]:
from forgather.project import Project
import forgather.nb.notebooks as nb
from pprint import pp

# Load default baseline config
proj = Project()

outputs = proj(["main", "model"])
pp(outputs)

# For easier access
trainer = outputs["main"]
model = outputs["model"]

{'main': Trainer(train_dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7ff2a59e3220>, test_dataloader=<torch.utils.data.dataloader.DataLoader object at 0x7ff2a594e050>, model=Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
), loss_fn=CrossEntropyLoss(), optimizer=SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
), epochs=5, batch_size=64, logging_steps=100),
 'model': Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)}

## Train Model

The trainer is started by simply calling it.

In [3]:
trainer()

  0%|                                                                                                         …

2024-08-15 07:04:43          100  0.11  train-loss: 2.281     
2024-08-15 07:04:44          200  0.21  train-loss: 2.26253   
2024-08-15 07:04:44          300  0.32  train-loss: 2.25173   
2024-08-15 07:04:45          400  0.43  train-loss: 2.2478    
2024-08-15 07:04:45          500  0.53  train-loss: 2.22504   
2024-08-15 07:04:46          600  0.64  train-loss: 2.20235   
2024-08-15 07:04:46          700  0.75  train-loss: 2.20056   
2024-08-15 07:04:47          800  0.85  train-loss: 2.18143   
2024-08-15 07:04:47          900  0.96  train-loss: 2.14116   


  0%|                                                                                                         …

2024-08-15 07:04:48          938  1.0   eval-loss:  2.14191   accuracy: 50.9
2024-08-15 07:04:48        1,000  1.07  train-loss: 2.13721   
2024-08-15 07:04:49        1,100  1.17  train-loss: 2.07268   
2024-08-15 07:04:49        1,200  1.28  train-loss: 2.08282   
2024-08-15 07:04:50        1,300  1.39  train-loss: 2.06193   
2024-08-15 07:04:50        1,400  1.49  train-loss: 2.05358   
2024-08-15 07:04:51        1,500  1.6   train-loss: 1.98428   
2024-08-15 07:04:51        1,600  1.71  train-loss: 1.91841   
2024-08-15 07:04:52        1,700  1.81  train-loss: 1.9116    
2024-08-15 07:04:52        1,800  1.92  train-loss: 1.83476   


  0%|                                                                                                         …

...

2024-08-15 07:04:59        2,814  3.0   eval-loss:  1.47434   accuracy: 60.8
2024-08-15 07:04:59        2,900  3.09  train-loss: 1.40874   
2024-08-15 07:05:00        3,000  3.2   train-loss: 1.5162    
2024-08-15 07:05:00        3,100  3.3   train-loss: 1.44716   
2024-08-15 07:05:01        3,200  3.41  train-loss: 1.37735   
2024-08-15 07:05:01        3,300  3.52  train-loss: 1.33895   
2024-08-15 07:05:02        3,400  3.62  train-loss: 1.35463   
2024-08-15 07:05:02        3,500  3.73  train-loss: 1.32036   
2024-08-15 07:05:03        3,600  3.84  train-loss: 1.38361   
2024-08-15 07:05:03        3,700  3.94  train-loss: 1.28251   


  0%|                                                                                                         …

2024-08-15 07:05:04        3,752  4.0   eval-loss:  1.22872   accuracy: 62.2
2024-08-15 07:05:05        3,800  4.05  train-loss: 1.20158   
2024-08-15 07:05:05        3,900  4.16  train-loss: 1.21234   
2024-08-15 07:05:06        4,000  4.26  train-loss: 1.01742   
2024-08-15 07:05:06        4,100  4.37  train-loss: 1.15856   
2024-08-15 07:05:07        4,200  4.48  train-loss: 1.22419   
2024-08-15 07:05:07        4,300  4.58  train-loss: 1.02989   
2024-08-15 07:05:08        4,400  4.69  train-loss: 1.06938   
2024-08-15 07:05:08        4,500  4.8   train-loss: 1.05031   
2024-08-15 07:05:09        4,600  4.9   train-loss: 1.12351   


  0%|                                                                                                         …

2024-08-15 07:05:10        4,690  5.0   eval-loss:  1.07941   accuracy: 63.7


## Construct and Train with Adam Optimizer

This just goes straight from config to train, using the 'adam.yaml' configuration, where we have replaced the SGD optimizer with Adam.

Take a look at the actual config definition to see what changes were required to accomplish this.

In [4]:
trainer = Project(config_name="adam.yaml")()
trainer()

  0%|                                                                                                         …

2024-08-15 07:05:48          100  0.11  train-loss: 0.62368   
2024-08-15 07:05:49          200  0.21  train-loss: 0.62747   
2024-08-15 07:05:49          300  0.32  train-loss: 0.34301   
2024-08-15 07:05:50          400  0.43  train-loss: 0.61325   
2024-08-15 07:05:50          500  0.53  train-loss: 0.2276    
2024-08-15 07:05:51          600  0.64  train-loss: 0.33854   
2024-08-15 07:05:51          700  0.75  train-loss: 0.65316   
2024-08-15 07:05:52          800  0.85  train-loss: 0.40484   
2024-08-15 07:05:53          900  0.96  train-loss: 0.31205   


  0%|                                                                                                         …

2024-08-15 07:05:53          938  1.0   eval-loss:  0.42341   accuracy: 84.5
2024-08-15 07:05:54        1,000  1.07  train-loss: 0.38436   
2024-08-15 07:05:54        1,100  1.17  train-loss: 0.41055   
2024-08-15 07:05:55        1,200  1.28  train-loss: 0.45073   
2024-08-15 07:05:55        1,300  1.39  train-loss: 0.24093   
2024-08-15 07:05:56        1,400  1.49  train-loss: 0.4748    
2024-08-15 07:05:57        1,500  1.6   train-loss: 0.20765   
2024-08-15 07:05:57        1,600  1.71  train-loss: 0.32071   
2024-08-15 07:05:58        1,700  1.81  train-loss: 0.60491   
2024-08-15 07:05:58        1,800  1.92  train-loss: 0.28574   


  0%|                                                                                                         …

...

2024-08-15 07:06:05        2,814  3.0   eval-loss:  0.37397   accuracy: 86.2
2024-08-15 07:06:06        2,900  3.09  train-loss: 0.40315   
2024-08-15 07:06:06        3,000  3.2   train-loss: 0.51675   
2024-08-15 07:06:07        3,100  3.3   train-loss: 0.34919   
2024-08-15 07:06:07        3,200  3.41  train-loss: 0.172     
2024-08-15 07:06:08        3,300  3.52  train-loss: 0.17076   
2024-08-15 07:06:09        3,400  3.62  train-loss: 0.2518    
2024-08-15 07:06:09        3,500  3.73  train-loss: 0.30597   
2024-08-15 07:06:10        3,600  3.84  train-loss: 0.30211   
2024-08-15 07:06:10        3,700  3.94  train-loss: 0.23603   


  0%|                                                                                                         …

2024-08-15 07:06:11        3,752  4.0   eval-loss:  0.37112   accuracy: 86.2
2024-08-15 07:06:12        3,800  4.05  train-loss: 0.25203   
2024-08-15 07:06:12        3,900  4.16  train-loss: 0.31518   
2024-08-15 07:06:13        4,000  4.26  train-loss: 0.14734   
2024-08-15 07:06:13        4,100  4.37  train-loss: 0.44975   
2024-08-15 07:06:14        4,200  4.48  train-loss: 0.27402   
2024-08-15 07:06:14        4,300  4.58  train-loss: 0.23364   
2024-08-15 07:06:15        4,400  4.69  train-loss: 0.09645   
2024-08-15 07:06:15        4,500  4.8   train-loss: 0.09581   
2024-08-15 07:06:16        4,600  4.9   train-loss: 0.29859   


  0%|                                                                                                         …

2024-08-15 07:06:17        4,690  5.0   eval-loss:  0.35356   accuracy: 87.1


## Run all Project Configurations

You can directly load the meta-config only and use it to find and iterate over all configurations in the project.

In [ ]:
from forgather.meta_config import MetaConfig

meta = MetaConfig()
for config_name, _ in meta.find_templates(meta.config_prefix):
    proj = Project(config_name=config_name)
    print(f"{ ' Starting ' + proj.config_name + ' ':-^60}")
    trainer = proj()
    trainer()